In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras.applications import ResNet101V2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Processing Dataset
 These steps are run only once to process the data. It is necessary becuase the dataset doesnt fit to ram and after saving it as tf.dataset object, tensorflow loads the data part by part when it processes. So in normal experimental part we do not use this part.

In [3]:
x_train = np.load('/content/drive/MyDrive/dl_project/Preprocess2/Train_Preprocessed2/preprocessed_tarining_batch_final5.npy')
x_train = x_train.reshape(-1, x_train.shape[2], x_train.shape[3], x_train.shape[4])

In [17]:
for i in range(0,60):
  data = tf.data.Dataset.from_tensor_slices(x_train[i*500:(i+1)*500])
  tf.data.Dataset.save(data, '/content/drive/MyDrive/dl_project/visual_train_dataset/x_train/preprocessed_tarining_batch_final5_'+str(i)+'.data')

In [18]:
y_train = np.load('/content/drive/MyDrive/dl_project/Preprocess2/Train_Preprocessed2/labels_tarining_batch_final5.npy')
y_train = np.repeat(y_train, 5, axis=0)
y_train = tf.data.Dataset.from_tensor_slices(y_train)
tf.data.Dataset.save(y_train, '/content/drive/MyDrive/dl_project/visual_train_dataset/y_train/labels_tarining_batch_final5.data')

In [3]:
x_val = np.load('/content/drive/MyDrive/dl_project/Preprocess2/Validation_Preprocessed2/preprocessed_validation_batch_final5.npy')
x_val = x_val.reshape(-1, x_val.shape[2], x_val.shape[3], x_val.shape[4])

In [4]:
for i in range(0,20):
  data = tf.data.Dataset.from_tensor_slices(x_val[i*500:(i+1)*500])
  tf.data.Dataset.save(data, '/content/drive/MyDrive/dl_project/visual_train_dataset/x_val/preprocessed_validation_batch_final5_'+str(i)+'.data')

In [5]:
y_val = np.load('/content/drive/MyDrive/dl_project/Preprocess2/Validation_Preprocessed2/labels_validation_batch_final5.npy')
y_val = np.repeat(y_val, 5, axis=0)
y_val = tf.data.Dataset.from_tensor_slices(y_val)
tf.data.Dataset.save(y_val, '/content/drive/MyDrive/dl_project/visual_train_dataset/y_val/labels_validation_batch_final5.data')


# Loading Dataset

In [3]:
batch_size = 32

In [4]:
# https://discuss.tensorflow.org/t/recommended-way-to-save-load-data-to-from-disk-to-tf-data-dataset/13983/3
training_dataset_x = tf.data.Dataset.load('/content/drive/MyDrive/dl_project/visual_train_dataset/x_train/preprocessed_tarining_batch_final5_0.data')
for i in range(1,60):
  x = tf.data.Dataset.load('/content/drive/MyDrive/dl_project/visual_train_dataset/x_train/preprocessed_tarining_batch_final5_'+str(i)+'.data')
  training_dataset_x = training_dataset_x.concatenate(x)
training_dataset_y = tf.data.Dataset.load('/content/drive/MyDrive/dl_project/visual_train_dataset/y_train/labels_tarining_batch_final5.data')
train_dataset = tf.data.Dataset.zip((training_dataset_x, training_dataset_y))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [5]:
validation_dataset_x = tf.data.Dataset.load('/content/drive/MyDrive/dl_project/visual_train_dataset/x_val/preprocessed_validation_batch_final5_0.data')
for i in range(1,20):
  x = tf.data.Dataset.load('/content/drive/MyDrive/dl_project/visual_train_dataset/x_val/preprocessed_validation_batch_final5_'+str(i)+'.data')
  validation_dataset_x = validation_dataset_x.concatenate(x)
validation_dataset_y = tf.data.Dataset.load('/content/drive/MyDrive/dl_project/visual_train_dataset/y_val/labels_validation_batch_final5.data')
val_dataset = tf.data.Dataset.zip((validation_dataset_x, validation_dataset_y))
val_dataset = val_dataset.batch(batch_size)

# Model Experiments

In [6]:
#https://keras.io/guides/making_new_layers_and_models_via_subclassing/

@tf.keras.utils.register_keras_serializable() # for saving and loading the model
class VisualModel(tf.keras.Model):
    def __init__(self):
        super(VisualModel, self).__init__()
        self.resnet = ResNet101V2(weights='imagenet', include_top=False)
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense_output = tf.keras.layers.Dense(5)

    def call(self, inputs):
        x = self.resnet(inputs)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense_output(x)

# Instantiate the model
model = VisualModel()

In [7]:
%load_ext tensorboard
log_dir = "/content/drive/MyDrive/dl_project/tensorboard-logs/visual/"
run_name = "resnet-128-64"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir+run_name, histogram_freq=1)
tensorboard_callback.set_model(model)
early_stop_callback  = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
early_stop_callback.set_model(model)
callbacks=tf.keras.callbacks.CallbackList([tensorboard_callback,early_stop_callback])

In [8]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.00001)
loss_fn = tf.keras.losses.MeanAbsoluteError()
epochs = 50
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=[tf.keras.metrics.MeanAbsoluteError()]
              )

In [9]:
model.fit(train_dataset, epochs=epochs, validation_data=val_dataset,callbacks=[tensorboard_callback,early_stop_callback])

Epoch 1/50
937/937 [==============================] - 696s 628ms/step - loss: 0.1960 - mean_absolute_error: 0.1960 - val_loss: 0.1265 - val_mean_absolute_error: 0.1265
Epoch 2/50
 17/937 [..............................] - ETA: 7:50 - loss: 0.1349 - mean_absolute_error: 0.1349

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/drive/MyDrive/dl_project/visual_model/model_weights/'+run_name+'.keras')

In [ ]:
#%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/dl_project/tensorboard-logs/visual/
